In [1]:
import time
from __future__ import division # safety with double division
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd
import numpy as np
import random


In [2]:
random.seed(1024)
Opt = SolverFactory("gurobi")

# Clustering for Centroids
In this linear programming optimization, it will be optimizing for the Centroids location relative to the fixed assignments;

### M1:
This LP will fix the assignment of the Centroids and optimize for the Lowest possible distance of $C_i$ 

In [3]:
M1 = AbstractModel()
M1.name = "Clustering Centroid-Distance LP"

### M2
This LP will fix the Centroid Locations and redo the assignement. 

In [4]:
M2 = AbstractModel()
M2.name = "Clustering Assignment-LP"

## Parameters
Both LP's have same starting parameter that characterize the problem.
- **d**: number of dimensions
- **n**: number of points to cluster
- **k**: number of clusters to generate

In [5]:
M1.NumberOfDimensions = Param(within=NonNegativeIntegers)
M1.NumberOfPoints = Param(within=NonNegativeIntegers)
M1.NumberOfClusters = Param(within=NonNegativeIntegers)

In [6]:
M2.NumberOfDimensions = Param(within=NonNegativeIntegers)
M2.NumberOfPoints = Param(within=NonNegativeIntegers)
M2.NumberOfClusters = Param(within=NonNegativeIntegers)

## Set
- **Dimension Index (D)**: Set consisting of all possible possible dimensions an arbitrary point i.e. $[x_1, x_2, x_3 ... x_d]$
- **Points (P)**: Set consisting of all indexes for Points in the system. $[p_1, p_2, p_3 ... p_n]$
- **Cluster Index (C)**: Set consisting of possible ClusterIndex. $[c_1, c_2, c_3 ... c_k]$

In [7]:
M1.DimensionIndex = RangeSet(1,M1.NumberOfDimensions)
M1.PointsIndex = RangeSet(1,M1.NumberOfPoints)
M1.ClusterIndex = RangeSet(1,M1.NumberOfClusters)

In [8]:
M2.DimensionIndex = RangeSet(1,M2.NumberOfDimensions)
M2.PointsIndex = RangeSet(1,M2.NumberOfPoints)
M2.ClusterIndex = RangeSet(1,M2.NumberOfClusters)

## Inputs
- **Point**: $P_{i,d}$ where $i$ $\in$ PointsIndex and $j$ $\in$ DimensionIndex 

In [9]:
M1.Point = Param(M1.PointsIndex,M1.DimensionIndex, within=Reals)

In [10]:
M2.Point = Param(M2.PointsIndex,M2.DimensionIndex, within=Reals)

## Possible Variables
- **Centroid**: $C_{i,d}$ where i $\in$ ClusterIndex and d $\in$ dimensionalIndex 
- **Assignment**: $A_{i,j}$ where i $\in$ pointsIndex and j $\in$ clusteringIndex
- **Slack Positive**: $S^{+}_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D
- **Slack Negative** $S^{-}_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D

### M1 Variables
- **Centroid**: $C_{i,d}$ where i $\in$ ClusterIndex and d $\in$ dimensionalIndex 
- **Slack Positive**: $S^{+}_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D
- **Skack Negative** $S^{-}_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D

In [11]:
M1.Centroid=Var(M1.ClusterIndex, M1.DimensionIndex, within=Reals)
M1.Slack_Plus = Var(M1.PointsIndex,M1.ClusterIndex,M1.DimensionIndex, within=NonNegativeReals)
M1.Slack_Minus = Var(M1.PointsIndex,M1.ClusterIndex,M1.DimensionIndex, within=NonNegativeReals)

### M2 Variables
- **Assignment**: $A_{i,j}$ where i $\in$ pointsIndex and j $\in$ clusteringIndex
- **Slack Positive**: $S^{+}_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D
- **Skack Negative** $S^{-}_{i,j,d}$ where i $\in$ P, j $\in$ C, and d $\in$ D

In [12]:
M2.Assignment = Var(M2.PointsIndex, M2.ClusterIndex, within=Binary)
M2.Slack_Plus = Var(M2.PointsIndex,M2.ClusterIndex,M2.DimensionIndex, within=NonNegativeReals)
M2.Slack_Minus = Var(M2.PointsIndex,M2.ClusterIndex,M2.DimensionIndex, within=NonNegativeReals)

### M1 Fixed Value

In [13]:
M1.Assignment=Param(M1.PointsIndex, M1.ClusterIndex, default=0, within=Binary, mutable=True)

### M2 Fixed Value

In [14]:
M2.Centroid=Param(M2.ClusterIndex, M2.DimensionIndex,default=0,within=Reals, mutable=True)

# Model M1
## Objective Function for M1
$$ \sum_{i \in Points}\sum_{j \in Clusters}\sum_{d \in Dimensions} (S^{+}_{i,j,d}+S^{-}_{i,j,d}) $$

In [15]:
def ObjectiveFunction(M):
    return sum(\
               (M.Slack_Plus[i,j,d]+M.Slack_Minus[i,j,d])\
               for i in M.PointsIndex \
               for j in M.ClusterIndex \
               for d in M.DimensionIndex)
M1.Distance = Objective(rule=ObjectiveFunction, sense=minimize)
    

### Constraint 1: Distance Constraint
Used to convert distance metric into 1-norm
$$0=A_{i,j}\cdot(P_{i,d}-C_{j,d})+(S^{+}_{i,j,d}-S^{-}_{i,j,d}) \qquad \forall i \in P, j\in C, d \in D $$

In [16]:
def DistanceConstraint(M,i,j,d):
    return 0 == M.Assignment[i,j]*(M.Point[i,d]-M.Centroid[j,d])+M.Slack_Plus[i,j,d]-M.Slack_Minus[i,j,d]
M1.Norm = Constraint(M1.PointsIndex, M1.ClusterIndex, M1.DimensionIndex, rule = DistanceConstraint)

# Model M2
## Objective Function for M2
$$ \sum_{i \in Points}\sum_{j \in Clusters}\sum_{d \in Dimensions} (S^{+}_{i,j,d}+S^{-}_{i,j,d}) $$

In [17]:
def ObjectiveFunction(M):
    return sum(\
               (M.Slack_Plus[i,j,d]+M.Slack_Minus[i,j,d])\
               for i in M.PointsIndex \
               for j in M.ClusterIndex \
               for d in M.DimensionIndex)
M2.Distance = Objective(rule=ObjectiveFunction, sense=minimize)

### Constraint 1: Distance Constraint
Used to convert distance metric into 1-norm
$$0=A_{i,j}\cdot(P_{i,d}-C_{j,d})+(S^{+}_{i,j,d}-S^{-}_{i,j,d}) \qquad \forall i \in P, j\in C, d \in D $$

In [18]:
def DistanceConstraint(M,i,j,d):
    return 0 == M.Assignment[i,j]*(M.Point[i,d]-M.Centroid[j,d])+M.Slack_Plus[i,j,d]-M.Slack_Minus[i,j,d]
M2.Norm = Constraint(M2.PointsIndex, M2.ClusterIndex, M2.DimensionIndex, rule = DistanceConstraint)

### Constraint 2: Non-Empty Cluster
$$ 1\leq \sum_{i \in P} A_{i,j} \qquad \forall j \in C $$

In [19]:
def NonEmptyCluster(M, j):
    return 1<=sum(M.Assignment[i,j] for i in M.PointsIndex)
M2.NonEmptyBalance = Constraint(M2.ClusterIndex, rule = NonEmptyCluster)

### Constraint 3: Singular Assignment 
$$ 1 = \sum_j A_{i,j} \qquad \forall i \in P$$

In [20]:
def SingularAssignment(M, i):
    return 1==sum(M.Assignment[i,j] for j in M.ClusterIndex)
M2.SingularAssignmentBalance = Constraint(M2.PointsIndex, rule = SingularAssignment)

## Create Problem and Solver Instance

In [21]:
folder  = "../Data/Iris/"
dat_location ="iris"
dat = folder+dat_location+".dat"

In [22]:
df = pd.read_csv("../Data/Iris/IRIS.csv")
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [23]:
df.sample(3)

,sepal_length,sepal_width,petal_length,petal_width,species
62,6.0,2.2,4.0,1.0,Iris-versicolor
113,5.7,2.5,5.0,2.0,Iris-virginica
13,4.3,3.0,1.1,0.1,Iris-setosa


In [24]:
start_time = time.time()
original_instance2 = M2.create_instance(dat)
clusters = original_instance2.NumberOfClusters.value
points = original_instance2.NumberOfPoints.value
dimensions = original_instance2.NumberOfDimensions.value

In [25]:
def seeding_A(instance):
    clusters = instance.NumberOfClusters.value
    points = instance.NumberOfPoints.value
    for x in range(1,points+1):
        instance.Assignment[x,((x-1)%clusters)+1]=1

In [26]:
def seeding_B(instance, df):
    clusters = instance.NumberOfClusters.value
    sliced_df = df.sample(clusters)
    print(sliced_df.values)
    for row in sliced_df.values:
        for centroid_id in range(clusters):
            for dim in range(dimensions):
                instance.Centroid[centroid_id+1,dim+1]=row[dim]

In [27]:
cluster_column = ["iteration", "index"]+["x_"+str(d) for d in range(1,dimensions+1)]
centroid_DF = pd.DataFrame(columns=cluster_column)

In [28]:
assignment_colum  = ["iteration","P_i","C_j"]
assignment_DF = pd.DataFrame(columns=assignment_colum)

In [29]:
#seeding_A(current_instance1)
seeding_B(original_instance2, df=df)
count=0

[[6.0 2.7 5.1 1.6 'Iris-versicolor']
 [6.2 2.9 4.3 1.3 'Iris-versicolor']
 [7.1 3.0 5.9 2.1 'Iris-virginica']]


In [30]:
past_instance2 = None

In [ ]:
while(True):
    median_columns = ["group"]+["x_"+str(d) for d in range(1,dimensions+1)]
    median_df = pd.DataFrame(columns =median_columns)
    
    Soln2 = Opt.solve(original_instance2)
    original_instance2.solutions.load_from(Soln2)
    
    print("Current_instance 2: ", count)
    print("Termination Condition was "+str(Soln2.Solver.Termination_condition))
    
    ##display(current_instance2)
    ## Assignment Record
    for i in range(1,points+1):
        for j in range(1,clusters+1):
            if(original_instance2.Assignment[i,j] == 1):
                arr = [count, i, j]
                assignment_DF.loc[len(assignment_DF)] = arr
                arr2 = [j]
                for d  in range(1,dimensions+1):
                    arr2.append(value(original_instance2.Point[i,d]))
                median_df.loc[len(median_df)] = arr2
    
    ## Centroid Location
    for j in range(1,clusters+1):
        arr = [count, j]
        for d  in range(1,dimensions+1):
            arr.append(value(original_instance2.Centroid[j,d]))
        centroid_DF.loc[len(centroid_DF)] = arr
        
    print(median_df.groupby("group").median())
    print(median_df.groupby("group").median().values)
    median_val= median_df.groupby("group").median().values
    median_val = np.sort(median_val)
    print(past_instance2)
    for row in median_val:
        for centroid_id in range(clusters):
            for dim in range(dimensions):
                original_instance2.Centroid[centroid_id+1,dim+1]=row[dim]
    
    if(past_instance2 is not None):
        if np.array_equiv(median_val, past_instance2):
            break
            
    past_instance2 = median_val
    
    
    count+=1
    print()

Current_instance 2:  0
Termination Condition was optimal
        x_1   x_2  x_3  x_4
group                      
1.0    6.00  3.00  4.5  1.4
2.0    5.75  3.05  4.1  1.3
3.0    5.70  3.00  4.2  1.3
[[6.   3.   4.5  1.4 ]
 [5.75 3.05 4.1  1.3 ]
 [5.7  3.   4.2  1.3 ]]
None

Current_instance 2:  1
Termination Condition was optimal
       x_1  x_2   x_3   x_4
group                      
1.0    5.7  3.0  4.20  1.30
2.0    5.8  3.0  4.20  1.30
3.0    5.9  3.0  4.65  1.45
[[5.7  3.   4.2  1.3 ]
 [5.8  3.   4.2  1.3 ]
 [5.9  3.   4.65 1.45]]
[[1.4  3.   4.5  6.  ]
 [1.3  3.05 4.1  5.75]
 [1.3  3.   4.2  5.7 ]]

Current_instance 2:  2
Termination Condition was optimal
        x_1   x_2   x_3  x_4
group                       
1.0    5.85  3.05  4.25  1.4
2.0    5.60  3.00  4.00  1.3
3.0    5.90  3.00  4.60  1.5
[[5.85 3.05 4.25 1.4 ]
 [5.6  3.   4.   1.3 ]
 [5.9  3.   4.6  1.5 ]]
[[1.3  3.   4.2  5.7 ]
 [1.3  3.   4.2  5.8 ]
 [1.45 3.   4.65 5.9 ]]

Current_instance 2:  3
Termination Condition w

Current_instance 2:  25
Termination Condition was optimal
        x_1  x_2   x_3  x_4
group                      
1.0    5.80  3.0  4.20  1.3
2.0    5.60  3.0  4.20  1.3
3.0    6.05  3.0  4.45  1.4
[[5.8  3.   4.2  1.3 ]
 [5.6  3.   4.2  1.3 ]
 [6.05 3.   4.45 1.4 ]]
[[1.4  3.   4.45 6.  ]
 [1.4  3.   4.5  5.7 ]
 [1.2  3.   4.1  5.8 ]]

Current_instance 2:  26
Termination Condition was optimal
        x_1   x_2   x_3  x_4
group                       
1.0    5.85  3.05  4.25  1.4
2.0    5.60  3.00  4.00  1.3
3.0    5.90  3.00  4.60  1.5
[[5.85 3.05 4.25 1.4 ]
 [5.6  3.   4.   1.3 ]
 [5.9  3.   4.6  1.5 ]]
[[1.3  3.   4.2  5.8 ]
 [1.3  3.   4.2  5.6 ]
 [1.4  3.   4.45 6.05]]

Current_instance 2:  27
Termination Condition was optimal
       x_1  x_2  x_3  x_4
group                    
1.0    5.6  3.1  4.0  1.2
2.0    5.8  3.0  4.5  1.4
3.0    5.8  3.0  4.5  1.3
[[5.6 3.1 4.  1.2]
 [5.8 3.  4.5 1.4]
 [5.8 3.  4.5 1.3]]
[[1.4  3.05 4.25 5.85]
 [1.3  3.   4.   5.6 ]
 [1.5  3.   4.6  5.9 ]]



Current_instance 2:  50
Termination Condition was optimal
        x_1   x_2   x_3  x_4
group                       
1.0    5.85  3.05  4.25  1.4
2.0    5.60  3.00  4.00  1.3
3.0    5.90  3.00  4.60  1.5
[[5.85 3.05 4.25 1.4 ]
 [5.6  3.   4.   1.3 ]
 [5.9  3.   4.6  1.5 ]]
[[1.3  3.   4.2  5.8 ]
 [1.3  3.   4.2  5.6 ]
 [1.4  3.   4.45 6.05]]

Current_instance 2:  51
Termination Condition was optimal
       x_1  x_2  x_3  x_4
group                    
1.0    5.6  3.1  4.0  1.2
2.0    5.8  3.0  4.5  1.4
3.0    5.8  3.0  4.5  1.3
[[5.6 3.1 4.  1.2]
 [5.8 3.  4.5 1.4]
 [5.8 3.  4.5 1.3]]
[[1.4  3.05 4.25 5.85]
 [1.3  3.   4.   5.6 ]
 [1.5  3.   4.6  5.9 ]]

Current_instance 2:  52
Termination Condition was optimal
       x_1  x_2   x_3  x_4
group                     
1.0    6.0  3.0  4.45  1.4
2.0    5.7  3.0  4.50  1.4
3.0    5.8  3.0  4.10  1.2
[[6.   3.   4.45 1.4 ]
 [5.7  3.   4.5  1.4 ]
 [5.8  3.   4.1  1.2 ]]
[[1.2 3.1 4.  5.6]
 [1.4 3.  4.5 5.8]
 [1.3 3.  4.5 5.8]]

Current_instance 

Current_instance 2:  75
Termination Condition was optimal
       x_1  x_2  x_3  x_4
group                    
1.0    5.6  3.1  4.0  1.2
2.0    5.8  3.0  4.5  1.4
3.0    5.8  3.0  4.5  1.3
[[5.6 3.1 4.  1.2]
 [5.8 3.  4.5 1.4]
 [5.8 3.  4.5 1.3]]
[[1.4  3.05 4.25 5.85]
 [1.3  3.   4.   5.6 ]
 [1.5  3.   4.6  5.9 ]]

Current_instance 2:  76
Termination Condition was optimal
       x_1  x_2   x_3  x_4
group                     
1.0    6.0  3.0  4.45  1.4
2.0    5.7  3.0  4.50  1.4
3.0    5.8  3.0  4.10  1.2
[[6.   3.   4.45 1.4 ]
 [5.7  3.   4.5  1.4 ]
 [5.8  3.   4.1  1.2 ]]
[[1.2 3.1 4.  5.6]
 [1.4 3.  4.5 5.8]
 [1.3 3.  4.5 5.8]]

Current_instance 2:  77
Termination Condition was optimal
        x_1  x_2   x_3  x_4
group                      
1.0    5.80  3.0  4.20  1.3
2.0    5.60  3.0  4.20  1.3
3.0    6.05  3.0  4.45  1.4
[[5.8  3.   4.2  1.3 ]
 [5.6  3.   4.2  1.3 ]
 [6.05 3.   4.45 1.4 ]]
[[1.4  3.   4.45 6.  ]
 [1.4  3.   4.5  5.7 ]
 [1.2  3.   4.1  5.8 ]]

Current_instance 2:  7

Current_instance 2:  100
Termination Condition was optimal
       x_1  x_2   x_3  x_4
group                     
1.0    6.0  3.0  4.45  1.4
2.0    5.7  3.0  4.50  1.4
3.0    5.8  3.0  4.10  1.2
[[6.   3.   4.45 1.4 ]
 [5.7  3.   4.5  1.4 ]
 [5.8  3.   4.1  1.2 ]]
[[1.2 3.1 4.  5.6]
 [1.4 3.  4.5 5.8]
 [1.3 3.  4.5 5.8]]

Current_instance 2:  101
Termination Condition was optimal
        x_1  x_2   x_3  x_4
group                      
1.0    5.80  3.0  4.20  1.3
2.0    5.60  3.0  4.20  1.3
3.0    6.05  3.0  4.45  1.4
[[5.8  3.   4.2  1.3 ]
 [5.6  3.   4.2  1.3 ]
 [6.05 3.   4.45 1.4 ]]
[[1.4  3.   4.45 6.  ]
 [1.4  3.   4.5  5.7 ]
 [1.2  3.   4.1  5.8 ]]

Current_instance 2:  102
Termination Condition was optimal
        x_1   x_2   x_3  x_4
group                       
1.0    5.85  3.05  4.25  1.4
2.0    5.60  3.00  4.00  1.3
3.0    5.90  3.00  4.60  1.5
[[5.85 3.05 4.25 1.4 ]
 [5.6  3.   4.   1.3 ]
 [5.9  3.   4.6  1.5 ]]
[[1.3  3.   4.2  5.8 ]
 [1.3  3.   4.2  5.6 ]
 [1.4  3.   4.45 

...

Current_instance 2:  175
Termination Condition was optimal
       x_1  x_2  x_3  x_4
group                    
1.0    5.6  3.1  4.0  1.2
2.0    5.8  3.0  4.5  1.4
3.0    5.8  3.0  4.5  1.3
[[5.6 3.1 4.  1.2]
 [5.8 3.  4.5 1.4]
 [5.8 3.  4.5 1.3]]
[[1.4  3.05 4.25 5.85]
 [1.3  3.   4.   5.6 ]
 [1.5  3.   4.6  5.9 ]]

Current_instance 2:  176
Termination Condition was optimal
       x_1  x_2   x_3  x_4
group                     
1.0    6.0  3.0  4.45  1.4
2.0    5.7  3.0  4.50  1.4
3.0    5.8  3.0  4.10  1.2
[[6.   3.   4.45 1.4 ]
 [5.7  3.   4.5  1.4 ]
 [5.8  3.   4.1  1.2 ]]
[[1.2 3.1 4.  5.6]
 [1.4 3.  4.5 5.8]
 [1.3 3.  4.5 5.8]]

Current_instance 2:  177
Termination Condition was optimal
        x_1  x_2   x_3  x_4
group                      
1.0    5.80  3.0  4.20  1.3
2.0    5.60  3.0  4.20  1.3
3.0    6.05  3.0  4.45  1.4
[[5.8  3.   4.2  1.3 ]
 [5.6  3.   4.2  1.3 ]
 [6.05 3.   4.45 1.4 ]]
[[1.4  3.   4.45 6.  ]
 [1.4  3.   4.5  5.7 ]
 [1.2  3.   4.1  5.8 ]]

Current_instance 2:

Current_instance 2:  200
Termination Condition was optimal
       x_1  x_2   x_3  x_4
group                     
1.0    6.0  3.0  4.45  1.4
2.0    5.7  3.0  4.50  1.4
3.0    5.8  3.0  4.10  1.2
[[6.   3.   4.45 1.4 ]
 [5.7  3.   4.5  1.4 ]
 [5.8  3.   4.1  1.2 ]]
[[1.2 3.1 4.  5.6]
 [1.4 3.  4.5 5.8]
 [1.3 3.  4.5 5.8]]

Current_instance 2:  201
Termination Condition was optimal
        x_1  x_2   x_3  x_4
group                      
1.0    5.80  3.0  4.20  1.3
2.0    5.60  3.0  4.20  1.3
3.0    6.05  3.0  4.45  1.4
[[5.8  3.   4.2  1.3 ]
 [5.6  3.   4.2  1.3 ]
 [6.05 3.   4.45 1.4 ]]
[[1.4  3.   4.45 6.  ]
 [1.4  3.   4.5  5.7 ]
 [1.2  3.   4.1  5.8 ]]

Current_instance 2:  202
Termination Condition was optimal
        x_1   x_2   x_3  x_4
group                       
1.0    5.85  3.05  4.25  1.4
2.0    5.60  3.00  4.00  1.3
3.0    5.90  3.00  4.60  1.5
[[5.85 3.05 4.25 1.4 ]
 [5.6  3.   4.   1.3 ]
 [5.9  3.   4.6  1.5 ]]
[[1.3  3.   4.2  5.8 ]
 [1.3  3.   4.2  5.6 ]
 [1.4  3.   4.45 

Current_instance 2:  225
Termination Condition was optimal
        x_1  x_2   x_3  x_4
group                      
1.0    5.80  3.0  4.20  1.3
2.0    5.60  3.0  4.20  1.3
3.0    6.05  3.0  4.45  1.4
[[5.8  3.   4.2  1.3 ]
 [5.6  3.   4.2  1.3 ]
 [6.05 3.   4.45 1.4 ]]
[[1.4  3.   4.45 6.  ]
 [1.4  3.   4.5  5.7 ]
 [1.2  3.   4.1  5.8 ]]

Current_instance 2:  226
Termination Condition was optimal
        x_1   x_2   x_3  x_4
group                       
1.0    5.85  3.05  4.25  1.4
2.0    5.60  3.00  4.00  1.3
3.0    5.90  3.00  4.60  1.5
[[5.85 3.05 4.25 1.4 ]
 [5.6  3.   4.   1.3 ]
 [5.9  3.   4.6  1.5 ]]
[[1.3  3.   4.2  5.8 ]
 [1.3  3.   4.2  5.6 ]
 [1.4  3.   4.45 6.05]]

Current_instance 2:  227
Termination Condition was optimal
       x_1  x_2  x_3  x_4
group                    
1.0    5.6  3.1  4.0  1.2
2.0    5.8  3.0  4.5  1.4
3.0    5.8  3.0  4.5  1.3
[[5.6 3.1 4.  1.2]
 [5.8 3.  4.5 1.4]
 [5.8 3.  4.5 1.3]]
[[1.4  3.05 4.25 5.85]
 [1.3  3.   4.   5.6 ]
 [1.5  3.   4.6  5.9 ]

Current_instance 2:  250
Termination Condition was optimal
        x_1   x_2   x_3  x_4
group                       
1.0    5.85  3.05  4.25  1.4
2.0    5.60  3.00  4.00  1.3
3.0    5.90  3.00  4.60  1.5
[[5.85 3.05 4.25 1.4 ]
 [5.6  3.   4.   1.3 ]
 [5.9  3.   4.6  1.5 ]]
[[1.3  3.   4.2  5.8 ]
 [1.3  3.   4.2  5.6 ]
 [1.4  3.   4.45 6.05]]

Current_instance 2:  251
Termination Condition was optimal
       x_1  x_2  x_3  x_4
group                    
1.0    5.6  3.1  4.0  1.2
2.0    5.8  3.0  4.5  1.4
3.0    5.8  3.0  4.5  1.3
[[5.6 3.1 4.  1.2]
 [5.8 3.  4.5 1.4]
 [5.8 3.  4.5 1.3]]
[[1.4  3.05 4.25 5.85]
 [1.3  3.   4.   5.6 ]
 [1.5  3.   4.6  5.9 ]]

Current_instance 2:  252
Termination Condition was optimal
       x_1  x_2   x_3  x_4
group                     
1.0    6.0  3.0  4.45  1.4
2.0    5.7  3.0  4.50  1.4
3.0    5.8  3.0  4.10  1.2
[[6.   3.   4.45 1.4 ]
 [5.7  3.   4.5  1.4 ]
 [5.8  3.   4.1  1.2 ]]
[[1.2 3.1 4.  5.6]
 [1.4 3.  4.5 5.8]
 [1.3 3.  4.5 5.8]]

Current_instan

Current_instance 2:  275
Termination Condition was optimal
       x_1  x_2  x_3  x_4
group                    
1.0    5.6  3.1  4.0  1.2
2.0    5.8  3.0  4.5  1.4
3.0    5.8  3.0  4.5  1.3
[[5.6 3.1 4.  1.2]
 [5.8 3.  4.5 1.4]
 [5.8 3.  4.5 1.3]]
[[1.4  3.05 4.25 5.85]
 [1.3  3.   4.   5.6 ]
 [1.5  3.   4.6  5.9 ]]

Current_instance 2:  276
Termination Condition was optimal
       x_1  x_2   x_3  x_4
group                     
1.0    6.0  3.0  4.45  1.4
2.0    5.7  3.0  4.50  1.4
3.0    5.8  3.0  4.10  1.2
[[6.   3.   4.45 1.4 ]
 [5.7  3.   4.5  1.4 ]
 [5.8  3.   4.1  1.2 ]]
[[1.2 3.1 4.  5.6]
 [1.4 3.  4.5 5.8]
 [1.3 3.  4.5 5.8]]

Current_instance 2:  277
Termination Condition was optimal
        x_1  x_2   x_3  x_4
group                      
1.0    5.80  3.0  4.20  1.3
2.0    5.60  3.0  4.20  1.3
3.0    6.05  3.0  4.45  1.4
[[5.8  3.   4.2  1.3 ]
 [5.6  3.   4.2  1.3 ]
 [6.05 3.   4.45 1.4 ]]
[[1.4  3.   4.45 6.  ]
 [1.4  3.   4.5  5.7 ]
 [1.2  3.   4.1  5.8 ]]

Current_instance 2:

In [ ]:
print("Time to complete:", time.time() - start_time)

In [ ]:
assignment_DF.to_csv(folder+dat_location+seeding+"_Assignment_1_NORM.csv")

In [ ]:
centroid_DF.to_csv(folder+dat_location+seeding+"_Centroid_1_NORM.csv")